In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model

# Przygotowanie danych do modelu UBM, jego trening oraz liczenie LDA.

In [ ]:
df = pd.read_csv('/kaggle/input/all-people-df/df')

# Ścieżka do folderu, w którym znajdują się katalogi z nagraniami osób.
file_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie nazwy podfolderów z powyższej ścieżki (są to ID nagranych osób).
subfolders = [f.name for f in os.scandir(file_path) if f.is_dir()]

# Sortuję ID nagranych osób (najpierw muszę zamienić ID na liczbę).
subfolders = sorted([int(item) for item in subfolders])
subfolders = np.array(subfolders)

df = df.loc[np.isin(np.array(df['ID']), subfolders)]

# Tworzę oddzielne ramki dla kobiet i mężczyzn.
df_woman = df[df['SEX'] == ' F '].reset_index(drop=True)
df_man = df[df['SEX'] == ' M '].reset_index(drop=True)

In [ ]:
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie podfoldery (ID osób) z głównego folderu z nagraniami.
subfolders = [f.name for f in os.scandir(folders_path) if f.is_dir()]
# Sortuję ID osób i zamieniam je z powrotem na stringi (ID muszą być w formie tekstowej).
subfolders = sorted([int(item) for item in subfolders])
subfolders = [str(item) for item in subfolders]

# Tworzę pełne ścieżki do folderów dla każdego ID.
paths_with_ID = [folders_path + '/' + subfolder for subfolder in subfolders]

# Tworzę dwie ramki danych do przechowywania ID i sumarycznej długości nagrań dla kobiet i mężczyzn.
data_frame_for_duration_woman = pd.DataFrame(columns=['ID', 'duration'])
data_frame_for_duration_man = pd.DataFrame(columns=['ID', 'duration'])

# Pętla przez wszystkie osoby, aby obliczyć sumaryczną długość nagrań.
for path_with_ID in paths_with_ID:

    # Zbieram ścieżki do folderów wewnątrz folderu danej osoby (podfoldery).
    paths_inside_ID = [f.name for f in os.scandir(path_with_ID) if f.is_dir()]

    # Tworzę pełne ścieżki do plików nagrań (plików .flac) dla każdego folderu wewnątrz ID.
    full_paths_to_files = [path_with_ID + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie pliki audio dla danej osoby.
    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Obliczam łączną długość nagrań danej osoby.
    duration_in_seconds = 0
    for file_for_ID in all_files_for_ID:
        # Otwieram plik audio za pomocą SoundFile i obliczam długość nagrania na podstawie liczby próbek i częstotliwości próbkowania.
        with sf.SoundFile(file_for_ID) as f:
            frames = len(f)  # Liczba próbek (frames)
            sample_rate = f.samplerate  # Częstotliwość próbkowania
        duration = frames / sample_rate  # Długość nagrania w sekundach
        duration_in_seconds = duration_in_seconds + duration  # Sumowanie długości wszystkich nagrań

    # Wyciągam ID osoby z pełnej ścieżki.
    ID = path_with_ID.split('/')[-1]
    # Tworzę nowy rekord z ID i sumaryczną długością nagrań.
    new_record = [ID, duration_in_seconds]
    
    # Sprawdzam, czy ID osoby należy do kobiet i dodaję dane do odpowiedniej ramki danych.
    if np.isin(ID, df_woman['ID']):
        data_frame_for_duration_woman.loc[len(data_frame_for_duration_woman)] = new_record
    else:
        data_frame_for_duration_man.loc[len(data_frame_for_duration_man)] = new_record

    # Wyświetlam postęp pętli.
    print(ID)
    clear_output(wait=True)

# Sortuję ramki danych według długości nagrań w kolejności malejącej.
data_frame_for_duration_woman = data_frame_for_duration_woman.sort_values(by='duration', ascending=False)
data_frame_for_duration_man = data_frame_for_duration_man.sort_values(by='duration', ascending=False)

data_frame_for_duration_woman.to_csv('data_frame_for_duration_woman.csv', index=False)
data_frame_for_duration_man.to_csv('data_frame_for_duration_man.csv', index=False)

# Wybieram 50 kobiet i 50 mężczyzn o najdłuższej sumarycznej długości nagrań.
top_50_man = data_frame_for_duration_man.head(50)
top_50_woman = data_frame_for_duration_woman.head(50)


8975


In [ ]:
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

top_50_man_paths = [folders_path + '/' + ID for ID in top_50_man['ID']]
top_50_woman_paths = [folders_path + '/' + ID for ID in top_50_woman['ID']]

top_50_man_and_woman = top_50_man_paths + top_50_woman_paths

with open("top_50_man_and_woman.pkl", "wb") as file:
    pickle.dump(top_50_man_and_woman, file)

In [ ]:
def split_audio_to_slices(path_to_files, seconds):
    
    # Przechodzę do katalogów wewnątrz folderu osoby (ID osoby).
    # Każdy folder wewnętrzny zawiera więcej podfolderów, które mogą zawierać nagrania.
    paths_inside_ID = [f.name for f in os.scandir(path_to_files) if f.is_dir()]

    # Tworzę pełne ścieżki do podfolderów, aby przejść do wszystkich plików nagrań dla danej osoby.
    full_paths_to_files = [path_to_files + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie ścieżki do plików audio danej osoby.
    # Każdy plik powinien mieć rozszerzenie `.flac`, a wszystkie pliki są przechowywane w zmiennej `all_files_for_ID`.

    all_files_for_ID = []
    
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Łączę wszystkie nagrania danej osoby w jedno bardzo długie nagranie.
    # Używam częstotliwości próbkowania 16kHz (standardowe dla nagrań mowy).
    sr = 16000
    combined_signals = np.array([])

    for file_for_ID in all_files_for_ID:
        signal, sr = librosa.load(file_for_ID, sr=sr)
        combined_signals = np.concatenate([combined_signals, signal])



    # Długie nagranie dzielę na  fragmenty o podanej długości.
    # Fragmenty, które mają mniej niż zadeklarowane długości nagrania (resztki na końcu nagrania), są pomijane.
    list_for_parts = []
    len_of_combined_signals = len(combined_signals)
    step = seconds * sr  # Ustawienie skoku na 5 sekund
    
    for i in np.arange(start=0, stop=len_of_combined_signals-step, step=step):
        list_for_parts.append(combined_signals[i:i+step].tolist())

    parts = np.array(list_for_parts)

    

    # Liczba filtrów melowych, które określają, ile "czapek" melowych zostanie użytych do przetwarzania sygnału.
    quantity_of_mel_filters = 64


    # Liczę mel-spektogramy dla którkich fragmentów nagrań
    mel_filters_value_list = []
    
    for i in range(len(parts)):
        mel_filters_value = librosa.feature.melspectrogram(y=parts[i], 
                                                                sr=16000, 
                                                                n_mels=quantity_of_mel_filters).T
        mel_filters_value_list.append(mel_filters_value)

    mel_filters_values = np.array(mel_filters_value_list)

    # Funkcja zwraca MFCC dla nagrań o długości jednej sekundy
    return mel_filters_values

In [ ]:
train_mel_filters_values_list = []
train_owner = []

# W pętli korzystam z wcześniej zdefiniowanej funkcji, która dzieli nagrania każdej z 100 osób
# (50 mężczyzn i 50 kobiet) na zestaw treningowy i testowy.

for i in range(0, 100):
    
    # Wywołuję funkcję split_audio_to_slices, aby podzielić nagrania danej osoby na zbiory treningowe i testowe.
    mel_filters_values = split_audio_to_slices(top_50_man_and_woman[i], seconds = 1)

    # Dodaję uzyskane dane mel-spektrogramy do odpowiednich list.
    train_mel_filters_values_list.extend(mel_filters_values)

    # Do listy train_owner i test_owner dodaję identyfikatory osób (i) tyle razy, ile jest nagrań.
    train_owner.extend([i] * len(mel_filters_values))

    # Wyświetlam numer osoby w pętli, aby śledzić postęp.
    print(i)
    clear_output(wait=True)

99


In [ ]:
# Liczę skaler aby sieć dobrze się wytrenowała

commmon_df_for_train = np.concatenate(train_mel_filters_values_list, axis=0)

scaler = StandardScaler()

scaler.fit(commmon_df_for_train)

X_train = [scaler.transform(one_audio) for one_audio in train_mel_filters_values_list]

X_train = np.array(X_train)
train_owner = np.array(train_owner)

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
# Wydzielam zbiór treningowy i walidacyjny do modelu
train_size = int(np.floor(len(X_train) * 0.8))

# Losowy wybór indeksów, które zostaną użyte jako dane treningowe (80% próbek).
index_of_train = np.random.choice(np.arange(0, len(X_train)), size=train_size, replace=False)

# Reszta indeksów (20% próbek będzie zbiorem walidacyjnym)
rest_of_index = ~np.isin(np.arange(0, len(X_train)), index_of_train)

X_valid = X_train[rest_of_index]
X_train = X_train[index_of_train]

valid_owner = train_owner[rest_of_index]
train_owner = train_owner[index_of_train]

In [ ]:
# Przekształcam dane treningowe (X_train), aby miały odpowiedni kształt dla sieci CNN.
# Dodaję nowy wymiar (1) na końcu, ponieważ sieci konwolucyjne oczekują wejść o formacie 4D:
# (liczba próbek, wysokość, szerokość, liczba kanałów). Tutaj mamy 1 kanał (monofoniczne nagrania).

X_train = X_train.reshape((X_train.shape[0], 
                           X_train.shape[1], 
                           X_train.shape[2], 
                           1))

# Przekształcam dane walidacyjne (X_valid) do tego samego formatu 4D co dane treningowe.
X_valid = X_valid.reshape((X_valid.shape[0], 
                           X_valid.shape[1], 
                           X_valid.shape[2], 
                           1))



# Konwertuję etykiety dla zestawu treningowego (train_owner) na postać one-hot encoding dla klasyfikacji wieloklasowej (100 klas).
# Używam funkcji to_categorical, aby zamienić numeryczne etykiety (ID osób) na macierze o rozmiarze [100], gdzie 
# każda wartość reprezentuje prawdopodobieństwo przynależności do danej klasy.
y_train = to_categorical(train_owner, num_classes=100)

# Podobnie konwertuję etykiety dla zestawu walidacyjnego na one-hot encoding.
y_valid = to_categorical(valid_owner, num_classes=100)

In [ ]:
np.save('X_train.npy', X_train)
np.save('X_valid.npy', X_valid)

np.save('y_train.npy', y_train)
np.save('y_valid.npy', y_valid)

In [2]:
X_train = np.load('/kaggle/input/data-to-train-mel-spectrogram/X_train.npy')
X_valid = np.load('/kaggle/input/data-to-train-mel-spectrogram/X_valid.npy')

y_train = np.load('/kaggle/input/data-to-train-mel-spectrogram/y_train.npy')
y_valid = np.load('/kaggle/input/data-to-train-mel-spectrogram/y_valid.npy')

In [ ]:
# Ustawienie ziarna losowości dla powtarzalnych wyników
np.random.seed(11)
tf.random.set_seed(11)
np.random.seed(11)

# Inicjalizacja modelu
model = models.Sequential()


model.add(layers.Input(shape=(32, 64, 1)))


model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())


model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.Dropout(0.5))


model.add(layers.Flatten())


model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))


model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))


model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))

# Warstwa bottleneck
model.add(layers.Dense(128, activation='linear', name = 'bottleneck'))


model.add(layers.Dense(100, activation='softmax'))  


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(
    X_train,    
    y_train,    
    epochs=20,  
    batch_size=32, 
    verbose = 1,
    validation_data=(X_valid, y_valid),  
    callbacks=[reduce_lr, early_stopping]  
)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_48 (Conv2D)              │ (None, 32, 64, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 32, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 32, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 32, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_61 (Dropout)            │ (None, 32, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 32, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_72          │ (None, 32, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_62 (Dropout)            │ (None, 32, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 32, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_73          │ (None, 32, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_23            │ (None, 16, 32, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ (None, 16, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 16, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_74          │ (None, 16, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_24            │ (None, 8, 16, 256)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ (None, 8, 16, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 8, 16, 512)     │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_75          │ (None, 8, 16, 512)     │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_25            │ (None, 4, 8, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_65 (Dropout)            │ (None, 4, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 19,063,844 (72.72 MB)

 Trainable params: 19,058,244 (72.70 MB)

 Non-trainable params: 5,600 (21.88 KB)

Epoch 1/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 93s 20ms/step - accuracy: 0.3160 - loss: 2.8505 - val_accuracy: 0.7817 - val_loss: 0.7593 - learning_rate: 0.0010
Epoch 2/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 62s 16ms/step - accuracy: 0.7513 - loss: 0.8751 - val_accuracy: 0.8714 - val_loss: 0.4450 - learning_rate: 0.0010
Epoch 3/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 61s 16ms/step - accuracy: 0.8240 - loss: 0.6147 - val_accuracy: 0.8780 - val_loss: 0.4345 - learning_rate: 0.0010
Epoch 4/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 62s 16ms/step - accuracy: 0.8623 - loss: 0.4764 - val_accuracy: 0.8710 - val_loss: 0.4708 - learning_rate: 0.0010
Epoch 5/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 61s 16ms/step - accuracy: 0.8814 - loss: 0.4147 - val_accuracy: 0.9327 - val_loss: 0.2383 - learning_rate: 0.0010
Epoch 6/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 62s 16ms/step - accuracy: 0.8990 - loss: 0.3503 - val_accuracy: 0.9418 - val_loss: 0.2074 - learning_rate: 0.0010
Epoch 7/20
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 63s 17ms/step - accura

In [14]:
model.save("model.h5")

# Model uczył się tylko przez 20 epok ponieważ po dopasowaniu parametrów zapomniałem puscić go na większej liczbie :(